In [38]:
from bs4 import BeautifulSoup as bs 
import requests
import pandas as pd
import numpy as np
from time import sleep
import pprint

In [43]:
#WITHOUT HEADERS WE WILL GET ERROR 403 
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
#LISTS VALUES FROM SCRAPE
movie_name = []
year = []
duration = [] 
rate =[]
picture_url = []
category = []
director = []
description = []

In [44]:
#movie_name,year,duration,category,rate,director,actors,picture_url
pages=[1,2,3,4,5,6,7,8,9,10]
for num_page in pages:
    imdb_movies_url = 'https://www.imdb.com/list/ls048276758/?sort=moviemeter,asc&st_dt=&mode=detail&page='+str(num_page)
    page = requests.get(imdb_movies_url,headers=headers)
    soup = bs(page.content,'html.parser')
    data_per_movie = soup.findAll('div', attrs = {'class': 'lister-item mode-detail'})
    #Must put sleep - without the request may crash
    sleep(4)
   
    for store in data_per_movie:
        if store.h3.a.text is not None:
            name = store.h3.a.text
            movie_name.append(name)
        else: movie_name.append(None)


        if store.h3.find('span', class_ = "lister-item-year text-muted unbold") is not None:
            year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text.replace('(','').replace(')','')
            year.append(year_of_release)
        else:year.append(None)

        if store.p.find('span', class_ = 'runtime') is not None:            
            time = int(store.p.find('span', class_ = 'runtime').text.replace(' min',''))
            duration.append(time)
        else: duration.append(None)

        if store.p.find('span', class_ = 'genre') is not None:     
            categories = store.p.find('span', class_ = 'genre').text.replace("\n", "").rstrip().split(',')
            category.append(categories)
        else:category.append(None)

        if store.find('span', class_ = "ipl-rating-star__rating") is not None:
            rating = float(store.find('span', class_ = "ipl-rating-star__rating").text)
            rate.append(rating)
        else:rate.append(None)

        #Success only the first director
        if store.findAll('p',class_ = 'text-muted text-small')[1] is not None:
            directors_sections = store.findAll('p',class_ = 'text-muted text-small')[1]
            if directors_sections.find('a') is not None:
                director_name = directors_sections.find('a').text
                director.append(director_name)
            else:director.append(None)
        else:director.append(None)

        if store.find('div', class_ = "lister-item-image ribbonize") is not None:
            pic_section = store.find('div', class_ = "lister-item-image ribbonize")
            if pic_section.find('img', class_='loadlate')['loadlate'] is not None:           
                pic_url = pic_section.find('img', class_='loadlate')['loadlate']
                picture_url.append(pic_url)
            else:picture_url.append(None)
        else:picture_url.append(None)

        if store.findAll('p')[1].text is not None:
            desc= store.findAll('p')[1].text
            desc = desc.strip()
            description.append(desc)
        else: description.append(None)

        
      #TRY-EXCEPT not suit here because we need to fill the values per section   
  
  


In [45]:
print(len(movie_name))
print(len(year))
print(len(duration))
print(len(category))
print(len(director))
print(len(rate))
print(len(picture_url))
print(list(description))

1000
1000
1000
1000
1000
1000
1000
['Over the course of several years, two convicts form a friendship, seeking consolation and, eventually, redemption through basic compassion.', 'The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son.', 'When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.', 'The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.', 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a C.E.O., but his tragic past may doom the project and his team to disaster.', 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Da

In [50]:
#creating a dataframe 
movie_list = pd.DataFrame({ "Movie Name": movie_name,\
                            "Year of Release" : year,\
                            "Duration (Min)": duration,\
                            "Genres": category,\
                            "Director" : director,\
                            "Movie Rating": rate,\
                            'Description':description,\
                            "Photo URL":picture_url                            
                            })

In [48]:
movie_list['Photo URL']

0      https://m.media-amazon.com/images/M/MV5BNDE3OD...
1      https://m.media-amazon.com/images/M/MV5BM2MyNj...
2      https://m.media-amazon.com/images/M/MV5BMTMxNT...
3      https://m.media-amazon.com/images/M/MV5BNGNhMD...
4      https://m.media-amazon.com/images/M/MV5BMjAxMz...
                             ...                        
995    https://m.media-amazon.com/images/M/MV5BOTMwMD...
996    https://m.media-amazon.com/images/M/MV5BMTE1MT...
997    https://m.media-amazon.com/images/M/MV5BOTgwMD...
998    https://m.media-amazon.com/images/M/MV5BZGRkMW...
999    https://m.media-amazon.com/images/M/MV5BOWNjYz...
Name: Photo URL, Length: 1000, dtype: object

In [51]:
movie_list.head(20)

,Movie Name,Year of Release,Duration (Min),Genres,Director,Movie Rating,Description,Photo URL
0,The Shawshank Redemption,1994,142,[Drama],Frank Darabont,9.3,"Over the course of several years, two convicts...",https://m.media-amazon.com/images/M/MV5BNDE3OD...
1,The Godfather,1972,175,"[Crime, Drama]",Francis Ford Coppola,9.2,The aging patriarch of an organized crime dyna...,https://m.media-amazon.com/images/M/MV5BM2MyNj...
2,The Dark Knight,2008,152,"[Action, Crime, Drama]",Christopher Nolan,9.0,When the menace known as the Joker wreaks havo...,https://m.media-amazon.com/images/M/MV5BMTMxNT...
3,Pulp Fiction,1994,154,"[Crime, Drama]",Quentin Tarantino,8.9,"The lives of two mob hitmen, a boxer, a gangst...",https://m.media-amazon.com/images/M/MV5BNGNhMD...
4,Inception,2010,148,"[Action, Adventure, Sci-Fi]",Christopher Nolan,8.8,A thief who steals corporate secrets through t...,https://m.media-amazon.com/images/M/MV5BMjAxMz...
5,The Lord of the Rings: The Fellowship of the Ring,2001,178,"[Action, Adventure, Drama]",Peter Jackson,8.9,A meek Hobbit from the Shire and eight compani...,https://m.media-amazon.com/images/M/MV5BN2EyZj...
6,A Quiet Place,2018,90,"[Drama, Horror, Sci-Fi]",John Krasinski,7.5,A family struggles for survival in a world whe...,https://m.media-amazon.com/images/M/MV5BMjI0MD...
7,The Florida Project,2017,111,[Drama],Sean Baker,7.6,"Set over one summer, the film follows precocio...",https://m.media-amazon.com/images/M/MV5BMjg4Zm...
8,Blade Runner 2049,2017,164,"[Action, Drama, Mystery]",Denis Villeneuve,8.0,Young Blade Runner K's discovery of a long-bur...,https://m.media-amazon.com/images/M/MV5BNzA1Nj...
9,The Silence of the Lambs,1991,118,"[Crime, Drama, Thriller]",Jonathan Demme,8.6,A young F.B.I. cadet must receive the help of ...,https://m.media-amazon.com/images/M/MV5BNjNhZT...


In [52]:
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Movie Name       1000 non-null   object 
 1   Year of Release  1000 non-null   object 
 2   Duration (Min)   1000 non-null   int64  
 3   Genres           999 non-null    object 
 4   Director         1000 non-null   object 
 5   Movie Rating     1000 non-null   float64
 6   Description      1000 non-null   object 
 7   Photo URL        1000 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 62.6+ KB


In [53]:
#save to excl just for verify by hand
movie_list.to_excel('movies_imdb.xlsx')

In [54]:
# save the data in csv format
movie_list.to_csv("movies_imdb.csv")